In [2]:
%matplotlib inline
from __future__ import division
import matplotlib.pyplot as plt
from astropy.io import fits
import numpy as np
from astropy.cosmology import FlatLambdaCDM
from astropy import units as u
from astropy.table import Table, hstack, vstack
from scipy.stats import ks_2samp, beta
from functools import reduce

import math
import mpl_style
import csv
import hex_scatter as hs

H0 = 70.
cosmo = FlatLambdaCDM(H0, Om0=0.3)
h = H0/100

In [3]:
#loading in data from lookup tables
hdulist4 = fits.open('../lookup_all_cut.fits')
all_data = hdulist4[1].data
halo_mass = all_data['halo_mass'] #in units of log(Mhalo/(Msun))
pfeatures = all_data['pfeatures']
psmooth = all_data['psmooth']
color = all_data['color']
ssfr = all_data['ssfr']
M_r = all_data['M_r']
mass = all_data['mass']
z = all_data['z']
centsat = all_data['centsat']
redblue = all_data['redorblue']
sfing = all_data['passiveorSFing']
#galaxy_num = all_data['galaxy_num']
#group_num = all_data['group_num']

group_list = {}
with open('../lookup_group_cut.csv') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        introw = [int(i) for i in row]
        group_list[introw[0]] = introw[1:]

In [5]:
#creates group and galaxy tables
group_nums = []
cent_ssfr = []
cent_sfing = []
cent_pfeat = []
cent_mr = []
cent_mass = []
halo_mass_group = []
sf_frac = []
spiral_frac = []
smooth_frac = []
median_sat_pfeat = []
spiral_half_frac = []
gal_num = []

cent_redblue = []

for group, group_gals in group_list.iteritems():
    if len(group_gals) > 1: #remove single galaxy groups
        has_central = False 
        sat_pfeat = []
        sat_spiral = 0
        sat_smooth = 0
        sat_sfing = 0
        sat_spiral_half = 0
        sat_num = 0
        
        for gal in group_gals: #now loop through these galaxies
            ind = np.where(all_data['galaxy_num'] == gal) #index to use in other arrays for this galaxy
            
            if centsat[ind[0][0]] == 1: #if its a central galaxy
                cent_ssfr.append(ssfr[ind[0][0]])
                cent_pfeat.append(pfeatures[ind[0][0]])
                cent_mr.append(M_r[ind[0][0]])
                halo_mass_group.append(halo_mass[ind[0][0]])
                cent_mass.append(mass[ind[0][0]])
                cent_redblue.append(redblue[ind[0][0]])
                cent_sfing.append(sfing[ind[0][0]])
                has_central = True
            else: #satellite galaxy
                sat_pfeat.append(pfeatures[ind[0][0]])
                sat_num += 1
                if pfeatures[ind[0][0]] >= 0.8: #featured
                    sat_spiral += 1
                if pfeatures[ind[0][0]] <= 0.2: #smooth
                    sat_smooth += 1
                if ssfr[ind[0][0]] > (-10 + 0.094*(M_r[ind[0][0]] + 15)): #star forming 
                    sat_sfing += 1
                if pfeatures[ind[0][0]] >= 0.5: #featured by alt defn
                    sat_spiral_half += 1

        if has_central == True:
            median_sat_pfeat.append(np.median(sat_pfeat))
            sf_frac.append(sat_sfing/sat_num)
            spiral_frac.append(sat_spiral/sat_num)
            spiral_half_frac.append(sat_spiral_half/sat_num)
            smooth_frac.append(sat_smooth/sat_num)
            gal_num.append(sat_num+1)
            group_nums.append(group)
            
            #active_line = -10 + 0.094*(cent_mr[-1] + 15)
            #if cent_ssfr[-1] >= active_line:
            #    cent_sfing.append(True)
            #else:
            #    cent_sfing.append(False)
            
group_table = Table([group_nums, gal_num, halo_mass_group, cent_ssfr, cent_sfing, cent_redblue, cent_pfeat, cent_mass, sf_frac, spiral_frac, smooth_frac, spiral_half_frac, median_sat_pfeat],
                    names=('group_num', 'n_gal', 'halo_mass', 'cent_ssfr', 'sfing_central', 'cent_redblue', 'cent_pfeat', 'cent_mass', 'sat_sf_frac', 'sat_spiral_frac', 'sat_smooth_frac', 'sat_spiral_half_frac', 'median_sat_pfeat'))


In [6]:
#add group data to individual galaxies in galaxy_data

n_gal = []
cent_ssfr = []
SFing_central = []
central_pfeat = []
cent_mass = []
sat_SF_frac = []
sat_spiral_frac = []
sat_smooth_frac = []
cent_redblue = []
median_sat_pfeat = []
sat_spiral_half_frac = []

for g in range(len(all_data)): #loop thru galaxies
    ind = np.where(group_table['group_num'] == all_data['group_num'][g])[0]
    if ind.size > 0:
        n_gal.append(group_table['n_gal'].data[ind][0])
        cent_ssfr.append(group_table['cent_ssfr'].data[ind][0])
        SFing_central.append(group_table['sfing_central'].data[ind][0])
        central_pfeat.append(group_table['cent_pfeat'].data[ind][0])
        cent_mass.append(group_table['cent_mass'].data[ind][0])
        sat_SF_frac.append(group_table['sat_sf_frac'].data[ind][0])
        sat_spiral_frac.append(group_table['sat_spiral_frac'].data[ind][0])
        sat_smooth_frac.append(group_table['sat_smooth_frac'].data[ind][0])
        cent_redblue.append(group_table['cent_redblue'].data[ind][0])
        median_sat_pfeat.append(group_table['median_sat_pfeat'].data[ind][0])
        sat_spiral_half_frac.append(group_table['sat_spiral_half_frac'].data[ind][0])
    
group_data_add = Table([n_gal, cent_ssfr, cent_redblue, SFing_central, central_pfeat, cent_mass, sat_SF_frac, sat_spiral_frac, sat_smooth_frac, sat_spiral_half_frac, median_sat_pfeat],
                                   names=('n_gal', 'cent_ssfr', 'cent_redblue', 'sfing_central', 'cent_pfeat', 'cent_mass', 'sat_sf_frac', 'sat_spiral_frac', 'sat_smooth_frac', 'sat_spiral_half_frac', 'median_sat_pfeat'))
                                   #dtype=('i4', 'f8', 'S8', 'S8', 'f8', 'f8', 'f8', 'f8', 'f8'))

all_data = Table(all_data)
galaxy_table = hstack([all_data, group_data_add], join_type='inner')

In [7]:
group_table.write("../group_table.fits", format='fits', overwrite=True)
galaxy_table.write("../galaxy_table.fits", format='fits', overwrite=True)